In [105]:
import os
import re
import dotenv
import numpy as np
from numpy import inf
import pandas as pd
from pprint import pprint

In [133]:
# Load environment variables
project_dir = os.path.join(os.path.abspath(''), os.pardir)
dotenv_path = os.path.join(project_dir, '.env')

dotenv.load_dotenv(dotenv_path)

True

In [90]:
df = pd.read_csv(os.path.join(os.getenv('SAVE_INTERIM_FILES_PATH'), 'daily_measures_weighted.csv'), encoding='UTF-8')
df_agg = pd.read_csv(os.path.join(os.getenv('SAVE_INTERIM_FILES_PATH'), 'agg_posts_lda_merged_results_with_keywords.csv'), encoding='UTF-8')

In [91]:
df_init_obs = df.groupby('post_id').first().reset_index().rename(columns = {
    'Impressions': 'init_imps',
    'Engagements': 'init_engs'}).loc[:, ["post_id", "init_imps", "init_engs"]]

In [92]:
df_init_obs["log_init_imps"] = np.log(df_init_obs["init_imps"])
df_init_obs["log_init_engs"] = np.log(df_init_obs["init_engs"])
df_init_obs.loc[df_init_obs["log_init_engs"] == -inf, "log_init_engs"] = 0.0001

A:\git_projects\machine-learning-project\venv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log


In [94]:
df_ready = pd.merge(
    df_init_obs,
    df_agg.loc[:, ["post_id", "t Impressions", "t Engagements", "class_weight", "Dominant_Topic", "Keywords"]].rename(columns={
        "t Impressions": "log_all_imps",
        "t Engagements": "log_all_engs"
    }),
    on = 'post_id',
    how = 'inner'
)
df_ready["log_all_engrate"] = df_ready["log_all_engs"]/df_ready["log_all_imps"]
df_ready.head()

,post_id,init_imps,init_engs,log_init_imps,log_init_engs,log_all_imps,log_all_engs,class_weight,Dominant_Topic,Keywords,log_all_engrate
0,0958b102f46bf30d0fd0b066164da0b7b7bed762c4d7b7...,6883.0,226.0,8.836810,5.420535,8.836810,5.420535,0.001724,8,"partner,let,factory,read,customer,offer,topic,...",0.613404
1,0e16a8e384fe24d1b936336efd87a978fd36ef241205a1...,51653.0,4811.0,10.852304,8.478660,10.852304,8.478660,0.001724,1,"system,risk,product,use,production,range,high,...",0.781277
2,149644004573512a361a5d89ab8c9003080a3af20fca36...,13296.0,399.0,9.495219,5.988961,9.495219,5.988961,0.001724,10,"technology,learn,great,offer,new,future,part,h...",0.630734
3,27df549b4d17af132fb603068a3cb64376aa9824193b8f...,4224.0,714.0,8.348538,6.570883,8.348538,6.570883,0.011423,10,"technology,learn,great,new,future,part,health,...",0.787070
4,2a0d916a2d052d0d7adc424c8cb6c2c8f8e6e5da04f546...,2813.0,59.0,7.942007,4.077537,7.942007,4.077537,0.001724,10,"learn,great,new,future,part,health,see,find,pr...",0.513414


In [135]:
df_ready.to_csv(os.path.join(os.getenv('PROCESSED_DATA_FILES'), 'df_train_test.csv'), encoding='UTF-8', index=False)